In [1]:
import pandas as pd
import numpy as np

In [2]:
# Input data
df = pd.read_csv("BTC_raw.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date").set_index("date")
print("Step 0 summary:")
print(f"- Shape: {df.shape}")
print(f"- Date range: {df.index.min().date()} -> {df.index.max().date()}")

Step 0 summary:
- Shape: (3253, 8)
- Date range: 2017-01-01 -> 2025-11-27


In [3]:
# Keep the last data for duplicated dates
dup_count = df.index.duplicated(keep=False).sum()
print(f"Duplicate date count: {dup_count}")

df = df[~df.index.duplicated(keep="last")]

print("Step 0 summary:")
print(f"- Shape after cleaning: {df.shape}")
print(f"- Date range: {df.index.min().date()} -> {df.index.max().date()}")


Duplicate date count: 0
Step 0 summary:
- Shape after cleaning: (3253, 8)
- Date range: 2017-01-01 -> 2025-11-27


In [4]:
btc_df  = df[["open", "high", "low", "close", "volume"]].copy()

spx_df  = df[["SP500"]].dropna()
gold_df = df[["GOLD"]].dropna()
dxy_df  = df[["DXY"]].dropna()

print(f"- BTC df shape:  {btc_df.shape}")
print(f"- SPX df shape:  {spx_df.shape}")
print(f"- GOLD df shape: {gold_df.shape}")
print(f"- DXY df shape:  {dxy_df.shape}")

- BTC df shape:  (3253, 5)
- SPX df shape:  (2239, 1)
- GOLD df shape: (2239, 1)
- DXY df shape:  (2242, 1)


In [5]:
def add_price_features(df, price_col, prefix):

    out = df.copy()    

    # Log price
    log_col = f"{prefix}_log_price"
    out[log_col] = np.log(out[price_col])

    # Returns
    for h in [1, 5, 20]:
        out[f"{prefix}_ret_{h}d"] = out[log_col].diff(h)

    # Volatility
    out[f"{prefix}_vol_20d"] = out[f"{prefix}_ret_1d"].rolling(20).std()

    # Momentum (price change over 10 days)
    out[f"{prefix}_mom_10d"] = out[log_col].diff(10)

    # Trend: moving averages
    for w in [10, 20, 50]:
        out[f"{prefix}_sma{w}"] = out[log_col].rolling(w).mean()

    return out

In [6]:
# Feature engineering
btc_feat  = add_price_features(btc_df,  "close",      "btc")
spx_feat  = add_price_features(spx_df,  "SP500",  "SP500").dropna()
gold_feat = add_price_features(gold_df, "GOLD", "GOLD").dropna()
dxy_feat  = add_price_features(dxy_df,  "DXY",  "DXY").dropna()

print("Step 2 summary:")
print(f"- BTC feat shape:   {btc_feat.shape}")
print(f"- SPX feat shape:   {spx_feat.shape}")
print(f"- GOLD feat shape:  {gold_feat.shape}")
print(f"- DXY feat shape:   {dxy_feat.shape}")

Step 2 summary:
- BTC feat shape:   (3253, 14)
- SPX feat shape:   (2190, 10)
- GOLD feat shape:  (2190, 10)
- DXY feat shape:   (2193, 10)


In [ ]:
def merge_macro(left, right, prefix):
    cols = [c for c in right.columns if c.startswith(prefix)]
    temp = right[cols].sort_index()

    merged = pd.merge_asof(
        left.sort_index(),
        temp,
        left_index=True,
        right_index=True,
        direction="backward"
    )
    return merged

In [8]:
# merge macro features onto BTC timeline
full = btc_feat.copy()

full = merge_macro(full, spx_feat,  "SP500")
full = merge_macro(full, gold_feat, "GOLD")
full = merge_macro(full, dxy_feat,  "DXY")

print("Step 3 summary:")
print(f"- Full feature df shape: {full.shape}")
print(f"- Date range: {full.index.min().date()} -> {full.index.max().date()}")


Step 3 summary:
- Full feature df shape: (3253, 44)
- Date range: 2017-01-01 -> 2025-11-27


In [9]:
# Set targets
full["btc_ret_1d_arith"] = full["close"].pct_change()
full["y_price_next"]        = full["close"].shift(-1)
full["y_log_price_next"]    = full["btc_log_price"].shift(-1)
full["y_ret_1d_next"]       = full["btc_ret_1d_arith"].shift(-1)
full["y_log_ret_1d_next"]   = full["btc_ret_1d"].shift(-1)   # btc_ret_1d is log return

target_cols = [
    "y_price_next",
    "y_log_price_next",
    "y_ret_1d_next",
    "y_log_ret_1d_next",
]

print("Step 4 summary:")
for c in target_cols:
    print(f"- NaNs in {c}: {full[c].isna().sum()}")

Step 4 summary:
- NaNs in y_price_next: 2
- NaNs in y_log_price_next: 2
- NaNs in y_ret_1d_next: 1
- NaNs in y_log_ret_1d_next: 3


C:\Users\katha\AppData\Local\Temp\ipykernel_39396\913752856.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  full["btc_ret_1d_arith"] = full["close"].pct_change()


In [10]:
# Drop NA
before = full.shape[0]
full_ml = full.dropna()
after = full_ml.shape[0]

print("Step 5 summary:")
print(f"- Rows before dropna: {before}")
print(f"- Rows after dropna:  {after}")
print(f"- Date range: {full_ml.index.min().date()} -> {full_ml.index.max().date()}")

Step 5 summary:
- Rows before dropna: 3253
- Rows after dropna:  3177
- Date range: 2017-03-15 -> 2025-11-24


In [11]:
# Helper functions
target_cols = [
    "y_price_next",
    "y_log_price_next",
    "y_ret_1d_next",
    "y_log_ret_1d_next",
]

def make_xy(df, target_col):
    X = df.drop(columns=target_cols)  # all features stay
    y = df[target_col]
    return X, y

In [12]:
full_ml = full_ml.rename(columns=lambda x: x[4:] if x.startswith("btc_") else x)
full_ml = full_ml.rename(columns={"ret_1d": "log_ret"})
full_ml = full_ml.rename(columns={"SP500_ret_1d": "SP500_lret"})
full_ml = full_ml.rename(columns={"GOLD_ret_1d": "GOLD_lret"})
full_ml = full_ml.rename(columns={"DXY_ret_1d": "DXY_lret"})


In [13]:
full_ml.to_csv("../BTC_data.csv")